<a href="https://colab.research.google.com/github/jotaperes/GeocodingReverse/blob/main/Geocoding_Reverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Geocoding Reverse

Recentemente, surgiu uma demanda no meu trabalho relacionada a necessidade de descobrir endereços a partir das coordenadas de um determinado objeto. Então, achei que seria um projeto interessante para desenvolver.

Após algumas tentativas e testes diversos, cheguei a um código bem enxuto que realiza de forma bem prática e fácil todo o processo.

Tive como referência as seguintes fontes:

https://geopy.readthedocs.io/en/stable/#

https://towardsdatascience.com/reverse-geocoding-in-python-a915acf29eb6#:~:text=Reverse%20Geocoding%20is%20an%20essential,features%20is%20called%20Reverse%20Geocoding.

In [1]:
!pip install tqdm

import geopandas as gpds #https://geopandas.org/en/stable/index.html
import pandas as pd
import geopy 
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter #Necessário para grandes quantidades de acessos, o que não é o caso presente, mas vale deixar por aqui.
import tqdm #biblioteca para visualizar barra de carregamento
from tqdm._tqdm_notebook import tqdm_notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ModuleNotFoundError: ignored

In [ ]:
#Testando o reverse e analisando como os dados são extraídos.

coordinates = '-22.8810148, -43.0967501'
location = locator.reverse(coordinates, language = 'pt-br')
location.raw

Testando o geopandas com o endereço do Barra Shopping presente no Google.
Cabe esclarecer que ele dá o endereço aproximado e que as bases são diferentes da API do Google.
Aparentemente está tudo certo.

In [ ]:
local = 'Av. das Américas, 4666 - Barra da Tijuca, Rio de Janeiro - RJ, 22640-102'
gpds.tools.geocode(local, provider = 'nominatim', user_agent = "assinaturasjoaopedro@gmail.com", country_bias = 'Brazil')

In [ ]:
#importando os dados
df = pd.read_excel('localizacoes.xlsx')
df.head()

O geopy apenas aceita os dados no formato f'{y},{x}'. Por isso, é preciso criar essa nova coluna que chamei de geom.


In [ ]:
df['geom'] = df["Y"].map(str) + ','+ df['X'].map(str)
df['geom'][0]

In [ ]:
locator = Nominatim(user_agent='XXXXXX', timeout=10) #definindo o user_agent novamente
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001) #definindo o ratelimiter

Aqui, eu criei um loop que basicamente faz a requisição da informação para cada linha de meu banco de dados e salva primeiro em algumas variáveis, que depois são concatenadas em uma lista. Essa lista posteriormente se transforma em um dataframe.

In [ ]:
endereco_completo=[]

for i in tqdm(range(len(df))):
    location = locator.reverse(df['geom'][i], language = 'pt-br')
    object_id = df['ID'][i]
    cidade = location.raw['address']['city']
    estado = location.raw['address']['state']
    geom = location.raw['lat'] + ',' + location.raw['lon']
    endereco_completo.append([cidade, estado, geom, location, object_id])


In [ ]:
df_endereco = pd.DataFrame(endereco_completo, columns=["cidade", "estado", "geom", "location", "id"])

df_endereco

#Enfim, o final

Realizo um merge com os dois bancos.

In [ ]:
df = df.merge(df_endereco, left_on='ID', right_on='id')

In [ ]:
df.head()